In [ ]:
# Needed Libraries
!pip install tensorflow
!pip install tflearn
import nltk
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import tflearn
import tensorflow as tf
import random

In [ ]:
# Reset the tflearn graph from earlier training
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
nltk.download('punkt')

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
# Load any of the benchmark dataset
data=json.load(open('/content/drive/MyDrive/data/HWU65.json')) # Put the path to the dataset where you saved it

In [ ]:
train_df = pd.DataFrame(data, columns=['text', 'label'])
test_df = pd.DataFrame(data['test'], columns=['text', 'label'])
val_df = pd.DataFrame(data['val'], columns=['text', 'label']) #Excempted for Banking77 Dataset
print("Len(train):", len(train_df))
print("Len(test):", len(test_df))
print("Len(val):", len(val_df))

In [ ]:
# Show label unique
print(np.unique(train_df["label"]))
total_label = int(len(np.unique(train_df["label"])))
print("\n ====>   Total: ", total_label)

In [ ]:
# Check NaN data
train_df.isna().sum()

In [ ]:
# Check balance data
train_df['label'].value_counts()[:50]

In [ ]:
# pick 30 samples each intents or labels. We only experimented on the stated sample size per label due to memeroy constraint and to make comparisons with other approach on same proportion
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_df['text'], train_df['label'], random_state=20, train_size=0.30, stratify=train_df['label'])

In [ ]:
# your new dataset of 30 samples for each intents or labels
train_df= pd.DataFrame({'text':train_x, 'label':train_y})

In [ ]:
documents = []
classes = []
words  = []
ignore_words = ['?']

#loop through each sentence in  your 30 samples for each labels collect them into a document container
for index, row in train_df.iterrows():
  for i in range(len(row['text'])):
    #tokenize each word in the pattern
    w = nltk.word_tokenize(row['text'])
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, row['label']))
    #add to our classes list
    if row['label'] not in classes:
      classes.append(row['label'])

#stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique stemmed words', words)

In [ ]:
#creating our training data
training = []
output = []
#create an empty array for our output
output_empty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents:
  #initialize our bag of words
  bag = []
  #list of tokenized words for the pattern
  pattern_words = doc[0]
  # stem each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create our bag od words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import time

# Start the timer
start_time = time.time()

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.embedding(net, input_dim=len(words), output_dim=10,  trainable=True, name="EmbeddingLayer")
# ELM Layer
net = tflearn.fully_connected(net, 128)
net = tflearn.activations.tanh(net)

# create attention layer
attention_probs = tflearn.fully_connected(net, 128, activation='softmax')
attention_mul = tf.multiply(net, attention_probs)

# add fully connected layer and output layer
net = tflearn.fully_connected(attention_mul, len(train_y[0]), activation='softmax')
#net = tflearn.regression(net)
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy', metric='default', learning_rate=0.001)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
hist=model.fit(train_x, train_y, n_epoch=200, batch_size=128, show_metric=True, validation_set=(test_x, test_y))

model.save('model.tflearn')

In [ ]:
# Stop the timer
end_time = time.time()

# Calculate the elapsed time
training_time = end_time - start_time
print("Training time: ", training_time, "seconds")

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
# preparing the test for F1, precision and recall scores computation
import sklearn.metrics
Y_pred = model.predict(test_x)
Y_pred_classes = np.argmax(Y_pred,axis = 1)

import numpy as np
test_y = np.argmax(test_y, axis=1)

In [ ]:
#calculate precision, recall and F1 score
import sklearn.metrics
sklearn.metrics.precision_recall_fscore_support(Y_pred_classes, test_y,  beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')